In [5]:
import re
import sqlite3
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed, LSTM


""" Custom Libs """
import Cleaner as c
import TokenMgmt as tm

In [9]:
epochs = 150
dropout = 0.2

run_model = False
model_file = ""

False

In [11]:
sqlite_file = '../../data/database/deeplearning.sqlite'
table_name  = 'tweets'
cd          = c.CleanData(sqlite_file, table_name)
q           ='SELECT * FROM {} WHERE Author == "simonlporter";'.format(table_name)

cd.set_table(q)
data = cd.get_clean_table()

inp_sequences, total_words = tm.get_sequence_of_tokens(list(data.CleanText.values))
predictors, label, max_sequence_len = tm.generate_padded_sequences(inp_sequences, total_words)

In [8]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 40, input_length = input_len))
    model.add(LSTM(256, return_sequences = True))
    model.add(LSTM(512), return_sequences = True)
    model.add(LSTM(128))
    model.add(Dropout(dropout))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer = 'adam')
    checkpointer = ModelCheckpoint(filepath='model' + '/single-user-model-{epoch:02d}.hdf5', verbose = 1)
    return(model, checkpointer)

In [ ]:
model, checkpointer = create_model(max_sequence_len, total_words)
model.summary()

In [ ]:
if run_model == True:
    if tf.test.is_gpu_available():
        model.fit(predictors, label, epochs = epochs, verbose = 1)
else:
    model.load_weights(model_file)